In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import shutil

# Configuration
DATA_DIR = r'C:\Users\Daffa Rinali\Documents\4. Tugas diluar Matkul\workshop kcv\FP2\dataset'
BATCH_SIZE = 32
IMAGE_SIZE = 224  # MobileNet standard input size
EPOCHS = 10
NUM_CLASSES = len(os.listdir(os.path.join(DATA_DIR, 'train')))  # Automatically detect number of classes


In [18]:

# Create model save directory
MODEL_SAVE_DIR = r'C:\Users\Daffa Rinali\Documents\4. Tugas diluar Matkul\workshop kcv\FP2\model'
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Data transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:

# Dataset preparation
train_dataset = datasets.ImageFolder(
    os.path.join(DATA_DIR, 'train'),
    train_transforms
)

val_dataset = datasets.ImageFolder(
    os.path.join(DATA_DIR, 'valid'),
    val_test_transforms
)

test_dataset = datasets.ImageFolder(
    os.path.join(DATA_DIR, 'test'),
    val_test_transforms
)

In [ ]:

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Model setup
def create_model():
    model = models.mobilenet_v2(pretrained=True)
    # Modify classifier for custom number of classes
    model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)
    return model

model = create_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
best_val_accuracy = 0.0

for epoch in range(EPOCHS):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    epoch_loss = running_loss / len(train_dataset)
    
    print(f'Epoch {epoch+1}/{EPOCHS}')
    print(f'Train Loss: {epoch_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%')

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), os.path.join(MODEL_SAVE_DIR, 'best_model.pth'))
        print('Best model saved!')


In [ ]:

# Testing
model.load_state_dict(torch.load(os.path.join(MODEL_SAVE_DIR, 'best_model.pth')))
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Inference example
from PIL import Image

def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = val_test_transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    class_names = train_dataset.classes
    return class_names[predicted[0]]

# Example usage
print("Predicted class:", predict_image('dr strange.jpeg'))